In [ ]:
import sycamore

# s3 file path to the SortBenchmark dataset
paths = "s3://aryn-datasets-us-east-1/sort_benchmark/index.html"


# Initializng sycamore which also initializes Ray underneath
context = sycamore.init()

# Creating a DocSet
docset = context.read.binary(paths, parallelism=1, binary_format="html")

In [ ]:
from sycamore.transforms.partition import HtmlPartitioner

# We are using UnstructuredPdfPartitioner to partion the documents.
# Sycamore supports pluggable partitioners for different formats.
docset = docset.partition(partitioner=HtmlPartitioner())

In [ ]:
from sycamore.transforms.schema import OpenAISchema, OpenAIPropertyExtractor
from sycamore.llms import OpenAIModels, OpenAI
import os


# We are using OpenAIEntityExtractor which utilizes OpenAI and gpt-3.5-turbo model.
# You can write your own EntityExtractor as well.

# Replace the "api-key" with your API Key.
openai = OpenAI(OpenAIModels.GPT_3_5_TURBO.value, api_key=os.environ.get("OPENAI_API_KEY"))

docset = docset.extract_schema(
            schema_extractor=OpenAISchema("BenchmarkRun", llm=openai)
        ).explode(

        ).extract_properties(
            property_extractor=OpenAIPropertyExtractor(llm=openai)
        )

In [ ]:
docset.limit(10).show()